<a href="https://colab.research.google.com/github/SrivenkateswaraRao/Research-Project/blob/master/Pest_MobileNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from os import listdir
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import files
files.upload() 

In [2]:
import CreatePestImageData as Imagedataset
import SupportingFunctions as support

In [ ]:
support.DownLoadFiles('1aXb25Wp9sG7mL6AQgYY9ElrGcERi95JU',drive)

In [12]:
support.Unzip('/content/GREENPATROL_DB_V3.0.zip','/content/Pests/Images')

In [13]:
support.fileCount('/content/Pests/Images')

4330


In [3]:
dataSet = Imagedataset.createImageDataset()

In [4]:
class_object = {0 : 'egg_wf' ,
                1 : 'egg_bt', 2 : 'egg_ta', 3 : 'wf', 4 : 'bt', 5 : 'ta', 6 : 'ta_maps', 7 : 'ta_larvae', 8 : 'ta_maps+larvae', 9 : 'tomato'}

In [ ]:
dataSet.prepare_dataset('/content/Pests/Labels' , class_object ,224, '/content/Pests/Images/',  test_size=11241 )

In [ ]:
listdir('/content/Pests/Labels')

In [19]:
(trainY , trainX) , (testY , testX ) = (dataSet.trainY , dataSet.trainX ) , (dataSet.testY , dataSet.testX )


In [8]:
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
import tensorflow as tf

Using TensorFlow backend.


In [24]:
base = VGG19(include_top=False, input_shape=(224, 224, 3))

In [15]:
pretrained_model = tf.keras.applications.MobileNetV2(input_shape=[224,224,3], include_top=False)
pretrained_model.trainable = False

model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])
# compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
flatten_1 (Flatten)          (None, 62720)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                627210    
Total params: 2,885,194
Trainable params: 627,210
Non-trainable params: 2,257,984
_________________________________________________________________


In [25]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(44964, 32, 32, 3)
(44964,)
(11241, 32, 32, 3)
(11241,)


In [21]:
from tensorflow.keras.utils import to_categorical
trainY = to_categorical(trainY)
testY = to_categorical(testY)

In [29]:
filepath="/content/drive/My Drive/Models/MobileNetV2/weights-{epoch:02d}-{accuracy:.3f}.hdf5"
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath, monitor='accuracy',
                               verbose=1, mode='max',save_best_only=True)

callbacks_list = [checkpoint]

In [23]:
tf.config.experimental_run_functions_eagerly(True)


In [ ]:
history = model.fit(trainX,trainY,verbose=1,epochs=20,validation_data=(testX,testY),callbacks=[checkpoint])

Epoch 1/20
1406/1406 [==============================] - ETA: 0s - loss: 5.9027 - accuracy: 0.6221
Epoch 00001: accuracy improved from -inf to 0.62212, saving model to /content/drive/My Drive/Models/MobileNetV2/weights-01-0.622.hdf5
1406/1406 [==============================] - 1543s 1s/step - loss: 5.9027 - accuracy: 0.6221 - val_loss: 5.8583 - val_accuracy: 0.6261
Epoch 2/20
1406/1406 [==============================] - ETA: 0s - loss: 5.9251 - accuracy: 0.6233
Epoch 00002: accuracy improved from 0.62212 to 0.62330, saving model to /content/drive/My Drive/Models/MobileNetV2/weights-02-0.623.hdf5
1406/1406 [==============================] - 1525s 1s/step - loss: 5.9251 - accuracy: 0.6233 - val_loss: 5.9808 - val_accuracy: 0.6166
Epoch 3/20
 181/1406 [==>...........................] - ETA: 20:10 - loss: 6.1759 - accuracy: 0.6112